# Imports

In [12]:
import os
import sys
import time
import math
import gzip
import json
import uuid
import itertools
import importlib
from pathlib import Path
import csv
import re
import numpy as np
import asyncpg
import glob
import gzip
import pandas as pd
import matplotlib.pyplot as plt

import edcompanion.edsm_api
from edcompanion.calctools import *

def prettyprint(item):
    print(json.dumps(item, indent=4, sort_keys=False))


In [3]:

pgsql_params = dict(
    dsn=os.getenv("PGSQL_URL"),
    server_settings={'search_path': "eddb"}
)
pgpool = await asyncpg.create_pool(**pgsql_params)

In [ ]:
print([ dict(R) for R in await pgpool.fetch("SELECT * FROM eddb.systems WHERE name = $1", "Ix")])

# Guardian Archeology

In [8]:
from itertools import accumulate, permutations, combinations, product
from sklearn.cluster import DBSCAN


In [9]:

importlib.reload(edcompanion.edsm_api)
from edcompanion.edsm_api import get_edsm_info, get_systems_in_sphere


### Loading the data

In [10]:
guardiandata_path = os.path.join(os.getcwd(), 'data', 'guardian')
guardiandata_files = {n.split(' - ')[2].split('.')[-2]:os.path.join(guardiandata_path, n) for n in os.listdir(guardiandata_path) if 'Canonn - Guardians' in n}


In [11]:
guardiandata = {}
for n,p in guardiandata_files.items():
    try:
        guardiandata[n] = pd.read_csv(p)
    except:
        pass
    

In [12]:
soi = ['Ruins','Structures']

In [13]:
inter_columns = list([i for i in accumulate([set(guardiandata['Guardian '+n].columns) for n in soi], lambda D1, D2: D1 & D2)][-1])
union_columns = list([i for i in accumulate([set(guardiandata['Guardian '+n].columns) for n in soi], lambda D1, D2: D1 | D2)][-1])

column_order_inter = {c:i for c,i in zip(guardiandata['Guardian Beacons'].columns, range(len(guardiandata['Guardian Beacons'].columns))) }
inter_columns = sorted(inter_columns, key=lambda I:column_order_inter.get(I,100))
union_columns = sorted(union_columns, key=lambda I:column_order_inter.get(I,100))

In [ ]:

soi_systems = {
    r['System Name']:dict(
        coord=np.asarray([r[c] for c in ['x', 'y', 'z']]),
        info= np.asarray([g] + [r[c] for c in inter_columns])
    )
    for g in soi for i, r in guardiandata['Guardian '+g].iterrows()
}

soi_systemnames = np.vstack([v['info'] for k,v in soi_systems.items()])
soi_coordinates = np.vstack([v['coord'] for k,v in soi_systems.items()])
print(soi_coordinates.shape, soi_systemnames.shape)

#### Caching EDSM info 

In [12]:
with open(os.path.join(os.getcwd(),'data','guardian','systems_db.json'), 'rt') as jsonfile:
    systems_db = json.load(jsonfile)

if not systems_db:
    systems_db ={S:get_edsm_info(S) for S in soi_systems}
    with open(os.path.join(os.getcwd(),'data','guardian','systems_db.json'), 'wt') as jsonfile:
        json.dump(systems_db, jsonfile, indent=3)

else:
    updates = {S:get_edsm_info(S) for S in soi_systems if S not in systems_db}
    if updates:
        systems_db.update(updates)
        with open(os.path.join(os.getcwd(),'data','guardian','systems_db.json'), 'wt') as jsonfile:
            json.dump(systems_db, jsonfile, indent=3)


In [13]:

for I in systems_db.values():
    for B in I['bodies']:
        if B['parents'] is None:
            B['parents'] = []

### Expanding data on bodies

In [ ]:
# Collect info on system and bodies
df_soi = pd.DataFrame(soi_systemnames, columns=['type']+inter_columns).set_index(['System Name', 'Body Name', 'type']).sort_index()
df_soi.head(5)

In [ ]:
df_soi['Distance To Arrival'].astype(float).plot(kind='hist', logx=True)

In [16]:
df_soi['DATlog'] = np.log10(df_soi['Distance To Arrival'].astype(float))


In [ ]:
df_soi['DATlog'].hist()

In [ ]:
df_soi['Body Sub Type'].value_counts()

In [ ]:
body_info = {}
systems_brief ={}

for row in df_soi.reset_index().iterrows():
    data_row = row[1]
    system_name = data_row['System Name'].strip()
    system = systems_db[system_name]
    body_name = f"{system_name} {data_row['Body Name'].strip()}"
    if body_name not in body_info:

        star_bodies = {
            v['bodyId']:len(v['parents']) if v['parents'] is not None else 0
            for v in system['bodies']
            if v['type']=='Star' 
        }
        main_star = [v['bodyId'] for v in system['bodies'] if v.get('isMainStar')][0]
        stars = set(star_bodies.keys())

        body = [B for B in system['bodies'] if B['name'].lower()==body_name.lower()][0]
        body_info[body_name] = body

        parent_star_id = list(
            set(
                [P.get('Star') for P in body['parents'] if 'Star' in P]) &
                stars
        )
        if not parent_star_id:
            parent_star_id = [B['bodyId'] for B in system['bodies'] if B['name'].lower()==system_name.lower()]

        if not parent_star_id:
            parent_stars = sorted(list(star_bodies.keys()), key=lambda x:star_bodies[x])
            print(f"Assuming parent for {body_name} ({body['bodyId']}) kis in {parent_stars}")
        else:
            parent_stars = sorted(parent_star_id, key=lambda x:star_bodies[x], reverse=True)

        body['parent_star_id'] = parent_stars[0]
        
        systems_brief[system_name] = dict(
            bodycount = system['bodyCount'],
            starcount = len(stars),
            mainstar = [v.get('subType','-')[0]  for v in system['bodies'] if v.get('isMainStar')  ],
            stars = '/'.join([v.get('subType','-')[0] for v in system['bodies'] if v['type']=='Star']),

        )

#prettyprint([v['parent_star_id'] for k,v in body_info.items()])

In [ ]:
gsbrief = pd.DataFrame.from_dict(systems_brief, orient='index')
gsbrief.info()

In [ ]:
gsbrief.describe()

In [ ]:
gsbrief['bodycount'].value_counts().head(25)

In [ ]:
gsbrief['bodycount'].hist()

In [ ]:
gsbrief['starcount'].value_counts()

In [ ]:
gsbrief['mainstar'].value_counts()

In [ ]:
gsbrief['stars'].value_counts().head(20)

### Geometric clustering

There is an obvious pattern in the systems that have been discovered

In [ ]:
soi_systemnames = np.asarray(
    [[g]+[r[c] for c in inter_columns] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()] )
soi_coordinates = np.asarray([[r[c] for c in ['x', 'y', 'z']] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()])
print(soi_coordinates.shape, soi_systemnames.shape)

In [ ]:
soi_systems = {r['System Name']:np.asarray([r[c] for c in ['x', 'y', 'z']]) for g in soi for i, r in guardiandata['Guardian '+g].iterrows()}
soi_systemnames = np.asarray(
    [[g]+[r[c] for c in inter_columns] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()] )
soi_coordinates = np.asarray([[r[c] for c in ['x', 'y', 'z']] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()])
print(soi_coordinates.shape, soi_systemnames.shape)

#### Clusters

The guardian systems appear clustered, mostly around points of interest along major galactic travel routes

In [29]:
from sklearn.cluster import DBSCAN


In [ ]:
coord_clusters = DBSCAN(eps=160, min_samples=5).fit(soi_coordinates)
#print(np.unique(coord_clusters.labels_))
#print(np.count_nonzero(np.less(coord_clusters.labels_,0)))
print([(l, np.count_nonzero(np.equal(coord_clusters.labels_,l))) for l in np.unique(coord_clusters.labels_)])


A few systems do not fall in any cluster

In [ ]:
outliers = np.copy(soi_coordinates[np.less(coord_clusters.labels_,0)])
outlier_systemnames = np.copy(soi_systemnames[np.less(coord_clusters.labels_,0)])

pd.DataFrame([[-1]+n.tolist()+c.tolist() for n,c in zip(outlier_systemnames, outliers)])

Write them out to fit the EDAstro custom marker format

In [32]:
with open(f"custom/guardian-clusters.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='red',
                text='\n'.join([f"{cn:20}: {v}" for cn, v in zip(['type'] + inter_columns,row[0])]),
                **{c:v for c,v in zip(['x','y','z'], row[1])}
            )
            for row in [(n,c) for n,c in zip(outlier_systemnames, outliers)]
        ] + [
            dict(
                pin='cyan',
                text=f'Center of cluster {row[0]} with {row[2]} guardian sites',
                **{c:v for c,v in zip(['x','y','z'], row[1])}
            )
            for row in [
                (l, np.round(np.mean(soi_coordinates[np.equal(coord_clusters.labels_,l)], axis=0),2).tolist(), np.count_nonzero(np.equal(coord_clusters.labels_,l)) ) 
                for l in np.unique(coord_clusters.labels_) 
                if not l < 0]
        ]), of, indent=3)
    


## Two line ...

In [33]:
import numpy as np

from scipy.interpolate import interp1d, LinearNDInterpolator
from edcompanion.calctools import *


In [34]:

bkgimage = 'data/visited-systems-indexedregions.png'
bkgimgurl= 'https://edastro.b-cdn.net/mapcharts/visited-systems-indexedregions.png'
bkgimage_scale = 10 # lightyear per pixel
origin=[9000, 7000]
plt_dpi=100

img = plt.imread(bkgimage)


In [35]:
# fig, ax = plt.subplots(figsize=(img.shape[1]/plt_dpi, img.shape[0]/plt_dpi))
# ax.imshow(img)

# fig.savefig('galaxy.png')


#### Clustering the clusters

We cluster the centers of previously found clusters and calculate a line for each 'line cluster' using 

In [ ]:
cluster_centers = np.asarray([
    np.round(np.mean(soi_coordinates[np.equal(coord_clusters.labels_,l)], axis=0),2).tolist() 
    for l in np.unique(coord_clusters.labels_) if l >= 0])
line_clusters = DBSCAN(eps=9999, min_samples=3).fit(cluster_centers)
print([(l, np.count_nonzero(np.equal(line_clusters.labels_,l))) for l in np.unique(line_clusters.labels_)])
guardian_lines = {l:cluster_centers[np.equal(line_clusters.labels_,l)] for l in np.unique(line_clusters.labels_)}

In [37]:
with open(f"custom/two-lines.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='yellow', text=f'line 0 {str(row)}',
                **{c:v for c,v in zip(['x','y','z'], row)}
            )
            for row in guardian_lines[0]
        ] + [
            dict(
                pin='green', text=f'line 1 {str(row)}',
                **{c:v for c,v in zip(['x','y','z'], row)}
            )
            for row in guardian_lines[1]
        ]), of, indent=3)

In [ ]:
{
    l:np.std(distance_point_to_line(cluster_centers[np.equal(line_clusters.labels_,l)], **line_from_points(cluster_centers[np.equal(line_clusters.labels_,l)])._asdict()))
    for l in np.unique(line_clusters.labels_)
}

#### Clustering on all systems

Alternatively we can cluster on all systems

In [39]:
import edcompanion.calctools
importlib.reload(edcompanion.calctools)
from edcompanion.calctools import *
from sklearn.cluster import DBSCAN

In [ ]:
line_clusters_ps = DBSCAN(eps=9999, min_samples=3).fit(soi_coordinates)
print([(l, np.count_nonzero(np.equal(line_clusters_ps.labels_,l))) for l in np.unique(line_clusters_ps.labels_)])

guardian_lines = {l:soi_coordinates[np.equal(line_clusters_ps.labels_,l)] for l in np.unique(line_clusters_ps.labels_)}

#### Calculate the lines

In [41]:
glines = {
    f"line_{l}":line_from_points(soi_coordinates[np.equal(line_clusters_ps.labels_,l)])._asdict()
    for l in np.unique(line_clusters_ps.labels_)
}

In [ ]:
with open(f"data/guardian/glines.json", 'wt') as of:
    json.dump({
        n:{p:a.tolist() for p,a in l.items()}
        for n,l in glines.items()
    },of,indent=3)
with open(f"data/guardian/glines.json", 'rt') as jsonfile:
    print(jsonfile.read())

Nu even kijken wat het scheelt

In [ ]:
list(np.unique(line_clusters.labels_))

In [ ]:
{
    l:np.std(distance_point_to_line(cluster_centers[np.equal(line_clusters.labels_,l)], **line_from_points(soi_coordinates[np.equal(line_clusters_ps.labels_,l)])._asdict()))
    for l in np.unique(line_clusters.labels_)
}

In [ ]:
{
    l:np.dot(cluster_centers[np.equal(line_clusters.labels_,l)], line_from_points(soi_coordinates[np.equal(line_clusters_ps.labels_,l)]).direction)
    for l in np.unique(line_clusters.labels_)
}

In [46]:
gsystem_lines = {
    l:line_from_points(soi_coordinates[np.equal(line_clusters_ps.labels_,l)])._asdict()
    for l in np.unique(line_clusters_ps.labels_)
}


In [ ]:
gsystem_lines

In [48]:
with open(f"custom/projected-clusters.json", 'wt') as of:
        json.dump(dict(markers=[
        dict(
            pin='yellow',
            text=f"cluster {l}, distance {round(distance_point_to_line(cc, **gsystem_lines[l]),1)}\ncentre \t{cc}\nprojected \t{project_point_on_line(cc, **gsystem_lines[l])}",
            **{c:round(v,1) for c,v in zip(['x','y','z'], list(project_point_on_line(cc, **gsystem_lines[l])))}
        )
        for l in np.unique(line_clusters.labels_) 
        for cc in cluster_centers[np.equal(line_clusters.labels_,l)]
        if not l < 0
    ]),of,indent=3)

#### Selective

In [49]:
glines = {
    l:line_from_points(soi_coordinates[np.equal(line_clusters_ps.labels_,l)])
    for l in np.unique(line_clusters.labels_)
}


In [ ]:
print(json.dumps({
        n:{p:a.tolist() for p,a in l.items()}
        for n,l in {
            f"line_{l}":glines[l]._asdict()
            for l in glines
        }.items()
    },indent=3))


In [ ]:
glp = {
    l:np.percentile(distance_point_to_line(soi_coordinates[np.equal(line_clusters_ps.labels_,l)], **glines[l]._asdict()), 70)
    for l in np.unique(line_clusters.labels_)
}
print(glp)

In [67]:
glmask = {
    l:np.less(distance_point_to_line(soi_coordinates[np.equal(line_clusters_ps.labels_,l)], **glines[l]._asdict()), glp[l])
    for l in np.unique(line_clusters.labels_)
}

In [ ]:
{
    l:np.count_nonzero(glmask[l])
    for l in glmask
}

In [ ]:
{
    l:np.std(distance_point_to_line(soi_coordinates[np.equal(line_clusters_ps.labels_,l)][glmask[l]], **glines[l]._asdict()))
    for l in glines
}

In [ ]:
{
    l:np.mean(distance_point_to_line(soi_coordinates[np.equal(line_clusters_ps.labels_,l)][glmask[l]], **glines[l]._asdict()))
    for l in glines
}

In [71]:
glines = {
    l:line_from_points(soi_coordinates[np.equal(line_clusters_ps.labels_,l)][glmask[l]])
    for l in np.unique(line_clusters.labels_)
}

In [ ]:
print(json.dumps({
        n:{p:a.tolist() for p,a in l.items()}
        for n,l in {
            f"line_{l}":glines[l]._asdict()
            for l in glines
        }.items()
    },indent=3))


In [ ]:
with open(f"data/guardian/glines.json", 'wt') as of:
    json.dump({
        n:{p:a.tolist() for p,a in l.items()}
        for n,l in {
            f"line_{l}":glines[l]._asdict()
            for l in glines
        }.items()
    },of,indent=3)

with open(f"data/guardian/glines.json", 'rt') as jsonfile:
    print(jsonfile.read())

### Intersect

In [74]:
from numpy import empty_like, dot, array
from edcompanion.edsm_api import get_systems_in_sphere, get_systems_in_cube
from edcompanion.calctools import *


In [75]:

def perp( a ) :
    b = empty_like(a)
    b[0] = -a[1]
    b[1] = a[0]
    return b

# line segment a given by endpoints a1, a2
# line segment b given by endpoints b1, b2
# return 
def seg_intersect(a1,a2, b1,b2) :
    da = a2-a1
    db = b2-b1
    dp = a1-b1
    dap = perp(da)
    denom = dot( dap, db)
    num = dot( dap, dp )
    return (num / denom.astype(float))*db + b1

def ppol(point, line_dir, line_mean):
    return np.dot(np.asarray(point)-line_mean, line_dir) * line_dir + line_mean


#### Scikit-spatial intersection

In [ ]:
try:
    with open(f"data/guardian/glines.json", 'rt') as jsonfile:
        glines = json.load(jsonfile)
    print(f"Load glines from disk...")
except:
    pass


In [ ]:
glines

In [78]:
from skspatial.objects import Line
from skspatial.plotting import plot_3d


In [85]:
import importlib
importlib.reload(edcompanion.calctools)
from edcompanion.calctools import *

In [ ]:
glines

In [ ]:


line_a = Line(
    point=[glines['line_0']['support'][0],0,glines['line_0']['support'][2]] ,
    direction=[glines['line_0']['direction'][0],0,glines['line_0']['direction'][2]]
)
line_b = Line(
    point=[glines['line_1']['support'][0],0,glines['line_1']['support'][2]] ,
    direction=[glines['line_1']['direction'][0],0,glines['line_1']['direction'][2]]
)

guardian_home = line_a.intersect_line(line_b)

guardian_home.tolist()

In [ ]:
project_point_on_line(guardian_home, **glines['line_0'])

In [ ]:
distance_point_to_line(guardian_home, **glines['line_0'])

In [ ]:
get_systems_in_cube(project_point_on_line(guardian_home, **glines['line_0']))

In [ ]:
project_point_on_line(guardian_home, **glines['line_1'])

In [ ]:
distance_point_to_line(guardian_home, **glines['line_1'])

In [ ]:
get_systems_in_cube(project_point_on_line(guardian_home, **glines['line_1']))

In [111]:
with open(f"custom/guardian-intersect.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='white', text=f"intersect {str(guardian_home)} on y=0\n{sorted(get_systems_in_cube(guardian_home), key=lambda S: S['bodyCount'], reverse=True)[0]['name']}",
                **{c:round(v) for c,v in zip(['x','y','z'], guardian_home.tolist())}
            )
            
        ]), of, indent=3)

In [ ]:
get_edsm_info('Prua Hypa MI-N b25-0')

In [ ]:
get_systems_in_sphere('Iorady JO-Z d13-0', radius=60)

In [ ]:
get_systems_in_sphere('Iorady EI-B d13-0', radius=60)

In [ ]:
get_systems_in_sphere('Iorady EI-B d13-0', radius=60)

#### Using a homebrew method with interp1d

In [92]:
line_0 = interp1d(guardian_lines[0][:,0], guardian_lines[0][:,2], fill_value='extrapolate')
line_1 = interp1d(guardian_lines[1][:,0], guardian_lines[1][:,2], fill_value='extrapolate')


In [93]:
x0 = [np.min(guardian_lines[0][:,0]), np.max(guardian_lines[0][:,0])]
z0 = line_0(x0)
x0,z0

p0 = np.column_stack((x0,z0))

In [ ]:
x1 = [np.min(guardian_lines[1][:,0]), np.max(guardian_lines[1][:,0])]
z1 = line_1(x1)
p1 = np.column_stack((x1,z1))

In [ ]:
p0,p1

In [ ]:
guardian_home = seg_intersect(p0[0],p0[1], p1[0],p1[1])
with open(f"custom/guardian-intersect.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='white', text=f'intersect {str(guardian_home)}',
                y=0,
                **{c:round(v) for c,v in zip(['x','z'], guardian_home.tolist())}
            )
            
        ]), of, indent=3)

In [ ]:
guardian_home

In [ ]:
from edcompanion.edsm_api import get_systems_in_cube

[ 
    get_systems_in_cube(system=[guardian_home[0], y, guardian_home[1]]) 
    for y in range(-2000,2000, 100)
]


In [ ]:
target_point = np.array([0,0])
target_point = np.array([ -22761.6875 , -3380.84375]) # DSSA Maerzenbecher

np.linalg.norm(np.cross(guardian_home-p1[0], p1[0]-target_point))/np.linalg.norm(guardian_home-p1[0])

In [ ]:
np.linalg.norm(np.cross(guardian_home-p0[0], p0[0]-target_point))/np.linalg.norm(guardian_home-p0[0])

In [ ]:
#p3=target_point
target_point = np.array([ -22761.6875 , -3380.84375]) # DSSA Maerzenbecher
#p1=guardian_home
#p2=p1[0]
#The line extending the segment is parameterized as p1 + t (p2 - p1).
#The projection falls where t = [(p3-p1) . (p2-p1)] / |p2-p1|^2
l2 = np.sum((guardian_home-p1[0])**2)
#if you need the point to project on line extention connecting p1 and p2
t = np.sum((target_point - guardian_home) * (p1[0] - guardian_home)) / l2

#if you need to ignore if p3 does not project onto line segment
if t > 1 or t < 0:
  print('p3 does not project onto p1-p2 line segment')

#if you need the point to project on line segment between p1 and p2 or closest point of the line segment
t = max(0, min(1, np.sum((target_point - guardian_home) * (p1[0] - guardian_home)) / l2))

projection1 = guardian_home + t * (p1[0] - guardian_home)

In [ ]:
#p3=target_point
target_point = np.array([  -15755.34375 , 107.0625 ])
#p1=guardian_home
#p2=p1[0]
#The line extending the segment is parameterized as p1 + t (p2 - p1).
#The projection falls where t = [(p3-p1) . (p2-p1)] / |p2-p1|^2
l2 = np.sum((guardian_home-p1[0])**2)
#if you need the point to project on line extention connecting p1 and p2
t = np.sum((target_point - guardian_home) * (p1[0] - guardian_home)) / l2

#if you need to ignore if p3 does not project onto line segment
if t > 1 or t < 0:
  print('p3 does not project onto p1-p2 line segment')

#if you need the point to project on line segment between p1 and p2 or closest point of the line segment
t = max(0, min(1, np.sum((target_point - guardian_home) * (p1[0] - guardian_home)) / l2))

projection2 = guardian_home + t * (p1[0] - guardian_home)

In [ ]:
projection2

In [ ]:
from edcompanion.edsm_api import get_systems_in_cube

[ 
    get_systems_in_cube(system=[target_point[0], y, target_point[1]]) 
    for y in range(-2000,2000, 100)
]


In [ ]:
#p3=target_point SYREADIAE JX-F C0
target_point = np.array([ -9529.4375, -7428.4375])
#p1=guardian_home
#p2=p1[0]
#The line extending the segment is parameterized as p1 + t (p2 - p1).
#The projection falls where t = [(p3-p1) . (p2-p1)] / |p2-p1|^2
l2 = np.sum((guardian_home-p1[0])**2)
#if you need the point to project on line extention connecting p1 and p2
t = np.sum((target_point - guardian_home) * (p1[0] - guardian_home)) / l2

#if you need to ignore if p3 does not project onto line segment
if t > 1 or t < 0:
  print('p3 does not project onto p1-p2 line segment')

#if you need the point to project on line segment between p1 and p2 or closest point of the line segment
t = max(0, min(1, np.sum((target_point - guardian_home) * (p1[0] - guardian_home)) / l2))

projection3 = guardian_home + t * (p1[0] - guardian_home)

In [ ]:

with open(f"custom/guardian-intersect.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='white', text=f'intersect {str(guardian_home)}',
                y=0,
                **{c:round(v) for c,v in zip(['x','z'], guardian_home.tolist())}
            ),
            dict(
                pin='red', text=f'project DSSA Maerzenbecher {str(projection1)}',
                y=0,
                **{c:round(v) for c,v in zip(['x','z'], projection1.tolist())}
            ),
            dict(
                pin='red', text=f'project Hemi Cuda {str(projection2)}',
                y=0,
                **{c:round(v) for c,v in zip(['x','z'], projection2.tolist())}
            ),
            dict(**{
                    "pin": "red",
                    "text": "Syreadiae JX-F c0 \n -9529.4375 / -64.5 / -7428.4375",
                    "y": -64.5,
                    "x": -9529.4375,
                    "z": -7428.4375
                }            
            )
            
        ]), of, indent=3)

In [ ]:
projection

## Beacons

In [ ]:
with open(f"custom/beacons.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='yellow',
                text=f"{row['System Name']:24} Body: {row['Body Name']} {row['Distance To Arrival']}\n{row['Guardian Structure System']}",
                **{c:row[c] for c in ['x','y','z']}
            )
            for i, row in guardiandata['Guardian Beacons'].iterrows()
        ] ), of, indent=3)

In [ ]:
list(guardiandata['Guardian Beacons'].iterrows())[0][1]

In [ ]:
guardiandata['Guardian Beacons']

In [ ]:
pd.DataFrame([ 
    [l]+n.tolist()+c.tolist() 
    for l in np.unique(coord_clusters.labels_) 
    for n,c in zip(
        soi_systemnames[np.equal(coord_clusters.labels_,l)], 
        soi_coordinates[np.equal(coord_clusters.labels_,l)] )
])

In [ ]:
pd.DataFrame([ 
    [l]+n.tolist()+c.tolist() 
    for l in np.unique(coord_clusters.labels_) 
    for n,c in zip(
        soi_systemnames[np.equal(coord_clusters.labels_,l)], 
        soi_coordinates[np.equal(coord_clusters.labels_,l)] )
])

In [ ]:
write_markers('guardian-clusters', dict(
    red=[
        dict(
            name=str(row[2]), type=str(row[1]), body=f"{row[3]} - {row[4]}",
            x=row[6], y=row[7], z=row[8], distance=row[5] ) 
        for index, row in pd.read_csv('data/Guardian - Ruins.csv').iterrows() 
    ],
))

In [ ]:

outlier_clusters = DBSCAN(eps=200, min_samples=2).fit(outliers)
#print(np.unique(outlier_clusters.labels_))
#print(np.count_nonzero(np.less(outlier_clusters.labels_,0)))
pd.DataFrame([(l, np.count_nonzero(np.equal(outlier_clusters.labels_,l))) for l in np.unique(outlier_clusters.labels_)]).set_index(0)

In [ ]:
really_lonely = np.copy(outliers[np.less(outlier_clusters.labels_,0)])
really_lonely_systemnames = np.copy(outlier_systemnames[np.less(outlier_clusters.labels_,0)])

#print(really_lonely, really_lonely_systemnames)
pd.DataFrame([n.tolist()+c.tolist() for n,c in zip(really_lonely_systemnames, really_lonely)])

In [ ]:

for n,c in zip(really_lonely_systemnames, really_lonely)

In [ ]:
write_markers('guardian-isolated', dict(
    red=[
        dict(name=str(row['System Name']), type=str(row['Type(s)']), body=str(row['Body'])) 
        for index, row in pd.read_csv('data/Guardian - Ruins.csv').iterrows() 
    ],
))

In [ ]:
really_lonely

In [ ]:
pd.DataFrame(outlier_systemnames)

In [ ]:
for n, df in guardiandata.items():
    print(f"{n} - {len(df)}\n{set(df.columns)}\n\n")

In [ ]:
soi = ['Ruins','Structures','Beacons']
inter_columns = list([i for i in accumulate([set(guardiandata['Guardian '+n].columns) for n in soi], lambda D1, D2: D1 & D2)][-1])
union_columns = list([i for i in accumulate([set(guardiandata['Guardian '+n].columns) for n in soi], lambda D1, D2: D1 | D2)][-1])

column_order_inter = {c:i for c,i in zip(guardiandata['Guardian Beacons'].columns, range(len(guardiandata['Guardian Beacons'].columns))) }
inter_columns = sorted(inter_columns, key=lambda I:column_order_inter.get(I,100))
union_columns = sorted(union_columns, key=lambda I:column_order_inter.get(I,100))

In [ ]:
pd.concat([guardiandata['Guardian '+n][inter_columns] for n in soi]).sort_values('Body Name')

In [ ]:
inter_columns
    

In [ ]:
l = [set(guardiandata['Guardian '+n].columns) for n in ['Ruins','Structures','Beacons']]

In [ ]:
l

# Dividing the Galaxy

In [ ]:
# get the extend of the explored galaxy
qx = await pgpool.fetchrow("SELECT min(c20x), max(c20x) from systems")
qy = await pgpool.fetchrow("SELECT min(c20y), max(c20y) from systems")
qz = await pgpool.fetchrow("SELECT min(c20z), max(c20z) from systems")
print([(R.get("min"), R.get("max"), R.get("max")-R.get("min")) for R in [qx, qy, qz]])

In [ ]:
# get the extend of the explored galaxy
qx = await pgpool.fetchrow("SELECT min(x), max(x) from systems")
qy = await pgpool.fetchrow("SELECT min(y), max(y) from systems")
qz = await pgpool.fetchrow("SELECT min(z), max(z) from systems")
print([(R.get("min"), R.get("max"), R.get("max")-R.get("min")) for R in [qx, qy, qz]])

In [ ]:
rx = 

In [ ]:
known_extend = [(-42213.8125, 40503.8125, 82717.625), (-29359.8125, 39518.34375, 68878.15625), (-23405.0, 65630.15625, 89035.15625)]

In [ ]:
counts = [(int(math.floor(v[2]/200))) for v in known_extend]
print(counts)

In [ ]:
counts[0] * counts[1] * counts[2]

In [ ]:
pow(len(range(-40000, 40000, 200)),3)/pow(100,3)

In [ ]:
Xr, *YZ = [(int(200*math.floor(R.get("min")/200)) , int(200*math.floor(R.get("max")/200))) for R in [qx, qy, qz]]

In [ ]:
len(range(*Xr, 200))

In [ ]:
Yr, *Z = YZ

In [ ]:
len(range(*Yr, 200))

In [ ]:
Zr, *T = Z

In [ ]:
len(range(*Zr, 200))

In [ ]:
data = []
count = 0
start = time.process_time()

for cy in range(-3000, 3000, 200): # n=30
    count += 1
    qr = await pgpool.copy_records_to_table("volumes", records=[(cx,cy,cz)  for cx in range(-30000, 30000, 200) for cz in range(-24000, 66000,200)])
    print(f"{count}/{30}\t{round(100*count/30,2)}%, {round(((30 - count) * (time.process_time() - start)/count),1)} remaining")



In [ ]:
cy=0

In [ ]:
[(cx,cy,cz)  for cx in range(-30000, 30000, 2000) for cz in range(-24000, 66000,2000)]

In [ ]:
len(range(-24000, 66000, 200)) * len(range(-30000, 30000, 200)) * len(range(-3000, 3000, 200)) / 1000

In [ ]:
len(range(-30000, 30000, 200))